In [5]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from finance_byu.summarize import summary

load_dotenv()  # reads .env file
SAVE_PATH = os.getenv("SAVE_PATH")

  Using cached finance_byu-0.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached finance_byu-0.2.0-py3-none-any.whl (16 kB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 24.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 63.0 MB/s  0:00:00 eta 0:00:01
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 80.1 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 63.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [finance_byu] [statsmodels]


In [ ]:
df = pd.read_feather(SAVE_PATH)

In [ ]:
df['logret'] = np.log(1 + df['ret'])
df['mom'] = df.groupby('permno')['logret'].rolling(6,6).sum().reset_index(drop=True)
df['mom'] = df.groupby('permno')['mom'].shift(2)

In [ ]:
df['prclag'] = df.groupby('permno')['prc'].shift()
df['melag'] = df.groupby('permno')['me'].shift()
df['cashtoasset'] = df["cash"] / df["assets"]
df['currenttoasset'] = df["current"] / df["assets"]
df.drop(columns=["cash", "current", "assets"])

df = df.query("mom == mom and prclag >= 5").reset_index(drop=True)

In [ ]:
df["mom_first"] = df.groupby('caldt')['mom'].transform(pd.qcut,5,labels=[str(k) for k in range(1, 6)])
df["cashtoasset_first"] = df.groupby('caldt')['cashtoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])
df["currenttoasset_first"] = df.groupby('caldt')['currenttoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])
df["cashtoasset_second"] = df.groupby(['caldt', 'mom_first'])['cashtoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])
df["currenttoasset_second"] = df.groupby(['caldt', 'mom_first'])['currenttoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])

In [ ]:
# Equal weighted, cash to asset ratio
port = df.groupby(['caldt','mom_first','cashtoasset_second'])['ret'].mean()
port = port.unstack(level=['mom_first','cashtoasset_second'])
print('Summary statistics from month to month, mom-cashasset:')
summary(port).round(4)

NameError: name 'df' is not defined